# HHL for Portfolio Optimization With Constraint

参考资料：
- [Quantum computational finance: quantum algorithm for portfolio optimization](https://arxiv.org/abs/1811.03975)
- [Solving linear systems of equations using HHL and its Qiskit implementation](https://learn.qiskit.org/course/ch-applications/solving-linear-systems-of-equations-using-hhl-and-its-qiskit-implementation)

In [1]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, QAOA, SamplingVQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit.result import QuasiDistribution
from qiskit_aer.primitives import Sampler
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_finance.data_providers import RandomDataProvider
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [24]:
# set number of assets (= number of qubits)
num_assets = 2
seed = 123

# Generate expected return and covariance matrix from (random) time-series
stocks = [("TICKER%s" % i) for i in range(num_assets)]
data = RandomDataProvider(
    tickers=stocks,
    start=datetime.datetime(2021, 11, 17),
    end=datetime.datetime(2021, 11, 27),
    seed=seed,
)
data.run()

R = data.get_period_return_mean_vector()
Pi = np.ones(len(R))
Sigma = data.get_period_return_covariance_matrix()


求解问题： 
$$
\begin{pmatrix}
0 & 0 & R^T \\
0 & 0 & \vec{1}^T\\
R & \vec{1} & \Sigma
\end{pmatrix}
\begin{pmatrix}
\eta \\
\theta \\
\vec{w}
\end{pmatrix}
=
\begin{pmatrix}
E \\
B \\
\vec{0}
\end{pmatrix}
$$

In [52]:
# mu = np.sum(np.abs(R))
E = np.abs(R)[0]*2
B = num_assets

In [53]:
ZeroA = np.zeros((2,2))

R_Pi=np.dstack((R, Pi))[0]

RT_PiT=np.vstack((R, Pi))

A = np.concatenate((np.vstack((ZeroA, R_Pi)), np.vstack((RT_PiT,Sigma))),axis=1)

b = np.array([E,B]+[0]*num_assets)

In [27]:
A

array([[ 0.00000000e+00,  0.00000000e+00,  1.06228854e-01,
         9.84161669e-04],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         1.00000000e+00],
       [ 1.06228854e-01,  1.00000000e+00,  8.82430345e-02,
        -6.93419959e-04],
       [ 9.84161669e-04,  1.00000000e+00, -6.93419959e-04,
         4.91610035e-04]])

In [8]:
A_ = np.identity(4)

b_ = np.array([2,2,2,1])

## 求解问题

In [28]:

import numpy as np
from linear_solvers import NumPyLinearSolver, HHL
matrix = A
vector = b

### 基于经典的线性方程组求解

In [29]:
classical_solution = NumPyLinearSolver().solve(matrix,
                                               vector/np.linalg.norm(vector))

In [30]:
print('classical state:', classical_solution.state*np.linalg.norm(vector))

classical state: [-1.69008912  0.00305016  2.         -0.        ]


规划到原始值

In [31]:
x = classical_solution.state*np.linalg.norm(vector)
print("投资策略: ",x[2:])

投资策略:  [ 2. -0.]


### 基于HHL的求解方法

In [32]:
naive_hhl_solution = HHL().solve(matrix, vector/np.linalg.norm(vector))

In [33]:
print('naive state:')
print(naive_hhl_solution.state)

naive state:
       ┌─────────────────┐┌──────┐        ┌─────────┐
q68_0: ┤0                ├┤6     ├────────┤6        ├
       │  circuit-522831 ││      │        │         │
q68_1: ┤1                ├┤7     ├────────┤7        ├
       └─────────────────┘│      │┌──────┐│         │
q69_0: ───────────────────┤0     ├┤5     ├┤0        ├
                          │      ││      ││         │
q69_1: ───────────────────┤1     ├┤4     ├┤1        ├
                          │  QPE ││      ││  QPE_dg │
q69_2: ───────────────────┤2     ├┤3     ├┤2        ├
                          │      ││      ││         │
q69_3: ───────────────────┤3     ├┤2 1/x ├┤3        ├
                          │      ││      ││         │
q69_4: ───────────────────┤4     ├┤1     ├┤4        ├
                          │      ││      ││         │
q69_5: ───────────────────┤5     ├┤0     ├┤5        ├
                          └──────┘│      │└─────────┘
  q70: ───────────────────────────┤6     ├───────────
               

判断HHL与经典方法的求解值的二阶范数。

In [34]:
print('classical Euclidean norm:', classical_solution.euclidean_norm)
print('naive Euclidean norm:', naive_hhl_solution.euclidean_norm)

classical Euclidean norm: 1.3019122732597515
naive Euclidean norm: 1.3006295438132165


获取HHL求解得到的最优解

In [35]:
from qiskit.quantum_info import Statevector

naive_sv = Statevector(naive_hhl_solution.state).data

In [41]:
len(naive_sv)

512

In [46]:
# if len(naive_sv)= 512 Extract vector components; 1 000 0000 00 00(bin) == 256 & 1 000 0000 00 01(bin) == 257 & 1 000 0000 00 10(bin) == 258 &1 000 0000 00 11(bin) == 259
index = int(len(naive_sv)/2)
naive_full_vector = np.array([naive_sv[index], naive_sv[index+1], naive_sv[index+2], naive_sv[index+3]])

print('naive raw solution vector:', naive_full_vector)

## if len(naive_sv) == 128
# naive_full_vector = np.array([naive_sv[64], naive_sv[65], naive_sv[66], naive_sv[67]])

# print('naive raw solution vector:', naive_full_vector)

naive raw solution vector: [-0.0465746 -3.03808507e-15j  0.00025542+2.77724785e-16j
  0.0546222 +3.91545041e-15j  0.00012395+5.26175425e-16j]


获取最终的结果

In [49]:
def get_solution_vector(solution, index):
    """Extracts and normalizes simulated state vector
    from LinearSolverResult."""
    solution_vector = Statevector(solution.state).data[index:index+4].real
    norm = solution.euclidean_norm
    return norm * solution_vector / np.linalg.norm(solution_vector)

print('full naive solution vector:', get_solution_vector(naive_hhl_solution,index))
print('classical state:', classical_solution.state)

full naive solution vector: [-0.84387606  0.00462787  0.98968881  0.00224579]
classical state: [-0.84031656  0.00151655  0.99440502 -0.        ]


求解HHL得到在投资数据

In [51]:
print("HHL得到的投资数据: ", get_solution_vector(naive_hhl_solution, index)*np.linalg.norm(vector))

HHL得到的投资数据:  [-1.69724818  0.00930781  1.99051451  0.00451685]
